In [890]:
import pandas as pd
import numpy as np
times = pd.read_excel('TIMES.xlsx')
times.set_index('Número', inplace = True)
times.fillna('NA', inplace = True)
times['Porcentagem de Vitorias'] = pd.Series(np.random.randn(len(list(times.Sigla))), index=times.index)
times_mundial_inicio = ['EDG','INF','DW','G2','SUP','ASC','C9','DFM','KBM','GRX','GMB','KLG','AFS','FW','PVB','RNG','VIT','GEN','KT','TL','MAD','FNC','IG','100']
times_mundial_grupos = ['AFS','FW','PVB','RNG','VIT','GEN','KT','TL','MAD','FNC','IG','100','EDG','G2','C9','GRX']
times_mundial_quartas = ['RNG','KT','FNC','IG','EDG','G2','C9','AFS']

In [891]:
def arrumadados(sheet,guia):
    arquivo = pd.ExcelFile(sheet)
    liga = pd.read_excel(arquivo,guia-1)
    liga = liga[3:]
    liga.fillna(0,inplace = True)
    liga.replace(0.7,0.8, inplace = True)
    return liga

vitorias= {}
jogos = {}
for sigla in list(times.Sigla):
    vitorias[sigla]=0
    jogos[sigla]=0

In [892]:
cblol1 = arrumadados('CBLOL_BRASIL.xlsx',1)
cblol2 = arrumadados('CBLOL_BRASIL.xlsx',2)
cls1 = arrumadados('CLS_LAS.xlsx',1)
cls2 = arrumadados('CLS_LAS.xlsx',2)
eu_lcs1 = arrumadados('EU_LCS_EUROPA.xlsx',1)
eu_lcs2 = arrumadados('EU_LCS_EUROPA.xlsx',2)
lck1 = arrumadados('LCK_COREIA.xlsx',1)
lck2 = arrumadados('LCK_COREIA.xlsx',2)
lcl1 = arrumadados('LCL_CIS.xlsx',1)
lcl2 = arrumadados('LCL_CIS.xlsx',2)
ljl1 = arrumadados('LJL_JAPAN.xlsx',1)
ljl2 = arrumadados('LJL_JAPAN.xlsx',2)
lln1 = arrumadados('LLN_LAN.xlsx',1)
lln2 = arrumadados('LLN_LAN.xlsx',2)
lms1 = arrumadados('LMS_TAIWAN.xlsx',1)
lms2 = arrumadados('LMS_TAIWAN.xlsx',2)
lpl1 = arrumadados('LPL_CHINA.xlsx',1)
lpl2 = arrumadados('LPL_CHINA.xlsx',2)
na_lcs1 = arrumadados('NA_LCS_EUA.xlsx',1)
na_lcs2 = arrumadados('NA_LCS_EUA.xlsx',2)
opl1 = arrumadados('OPL_OCEANIA.xlsx',1)
opl2 = arrumadados('OPL_OCEANIA.xlsx',2)
seatour1 = arrumadados('SEATOUR_SEA.xlsx',1)
seatour2 = arrumadados('SEATOUR_SEA.xlsx',2)
tcl1 = arrumadados('TCL_TURQUIA.xlsx',1)
tcl2 = arrumadados('TCL_TURQUIA.xlsx',2)
vcs1 = arrumadados('VCS_VIETNAN.xlsx',1)
vcs2 = arrumadados('VCS_VIETNAN.xlsx',2)
msi1 = arrumadados('MSI.xlsx',1)
msi2 = arrumadados('MSI.xlsx',2)
msi3 = arrumadados('MSI.xlsx',3)
msi4 = arrumadados('MSI.xlsx',4)
mundial1 = arrumadados('MUNDIAL_2018.xlsx',1)
mundial2 = arrumadados('MUNDIAL_2018.xlsx',2)
mundial3 = arrumadados('MUNDIAL_2018.xlsx',3)
teste = arrumadados('testando.xlsx',1)

In [893]:
def elo(liga,maximo,k=8):
    tam=int(liga.shape[1]/5)
    for semana in range(1,tam+1):
        somar = {}
        for sigla in list(times.Sigla):
            somar[sigla]=0
        for time in list(times.Sigla):
            for perdedorrr in list(liga['Perdedor'+str(semana)]):
                if perdedorrr!=0:
                    if time==perdedorrr:
                        jogos[perdedorrr]+=1
            for vencedor in list(liga['Vencedor'+str(semana)]):
                if vencedor!=0:
                    if time==vencedor:
                        jogos[vencedor]+=1
                        vitorias[vencedor]+=1
                        dados_lista = liga[liga['Vencedor'+str(semana)]==vencedor]
                        for num_perd in range(0,len(list(dados_lista['Perdedor1']))):
                            perdedor = list(dados_lista['Perdedor'+str(semana)])[num_perd]
                            peso = float(list(dados_lista['Peso'+str(semana)])[num_perd])
                            parametro_vencedor = 10**(times[times.Sigla==vencedor]['Força atualizada'].values/400)
                            parametro_perdedor = 10**(times[times.Sigla==perdedor]['Força atualizada'].values/400)
                            prob_ganhador = parametro_vencedor/(parametro_vencedor+parametro_perdedor)
                            prob_perdedor = parametro_perdedor/(parametro_vencedor+parametro_perdedor)
                            
                            if times[times.Sigla==vencedor]['Força atualizada'].values >= maximo:
                                somar[vencedor]+= k/2*(peso - prob_ganhador)
                            else:
                                somar[vencedor]+= k*(peso - prob_ganhador)
                            if times[times.Sigla==perdedor]['Força atualizada'].values >= maximo:
                                somar[perdedor]-= k/2*(prob_perdedor)
                            else:
                                somar[perdedor]-= k*(prob_perdedor)
                else:
                    pass
        for time in list(times.Sigla):
            ind = times[times.Sigla==time].index
            times.loc[ind,'Força atualizada']=times.loc[ind,'Força atualizada']+somar[time]
        



def previsao(timeA,timeB):
    parametro_timeA = 10**(times[times.Sigla==timeA]['Força atualizada'].values/400)
    parametro_timeB = 10**(times[times.Sigla==timeB]['Força atualizada'].values/400)
    prob_timeA = parametro_timeA/(parametro_timeA+parametro_timeB)
    prob_timeB = parametro_timeB/(parametro_timeA+parametro_timeB)
    print('A probabilidade de {} ganhar é de {}%'.format(timeA,prob_timeA))
    print('A probabilidade de {} ganhar é de {}%'.format(timeB,prob_timeB))
    
def por_vit():
    porcentagem_de_vitorias = {}
    for time in list(times.Sigla):
        porcentagem_de_vitorias[time] = vitorias[time]/jogos[time]
    for time in list(times.Sigla):
        times.replace(times[times.Sigla==time]['Porcentagem de Vitorias'].values,porcentagem_de_vitorias[time], inplace = True)
def elasticidade(a1,a2,b,c1,c2,regiao):
    elo(a1,b,c1)
    elo(a2,b,c2)
    quad = 0
    lista_a = list(times[times.Região==regiao].sort_values(by=['Força atualizada'],ascending = False).round(2).index)
    lista_b = list(times[times.Região==regiao].index)
    for i in range(0,len(lista_a)):
        quad+= (lista_a[i] - lista_b[i])**(2)
    return quad/len(lista_a)

In [894]:
maximos = [1800,1850,1900,1950,2000,10000]
porc = [0.5,1,1.5,2,2.5,3]
lista_de_valores = []
for KR in range(5,46):
    for MM in maximos:
        for KP in porc:
            import pandas as pd
            import numpy as np
            times = pd.read_excel('TIMES.xlsx')
            times.set_index('Número', inplace = True)
            times.fillna('NA', inplace = True)
            times['Porcentagem de Vitorias'] = pd.Series(np.random.randn(len(list(times.Sigla))), index=times.index)
            times_mundial_inicio = ['EDG','INF','DW','G2','SUP','ASC','C9','DFM','KBM','GRX','GMB','KLG','AFS','FW','PVB','RNG','VIT','GEN','KT','TL','MAD','FNC','IG','100']
            times_mundial_grupos = ['AFS','FW','PVB','RNG','VIT','GEN','KT','TL','MAD','FNC','IG','100','EDG','G2','C9','GRX']
            times_mundial_quartas = ['RNG','KT','FNC','IG','EDG','G2','C9','AFS']

            lms1 = arrumadados('LMS_TAIWAN.xlsx',1)
            lms2 = arrumadados('LMS_TAIWAN.xlsx',2)

            VALOR = elasticidade(lms1,lms2,MM,KR,KP*KR,'TWHKMA')
            if VALOR not in lista_de_valores:    
                lista_de_valores.append(VALOR)
            justica = list(times[times.Região=='TWHKMA'].sort_values(by=['Força atualizada'],ascending = False).round(2).Sigla)
            if VALOR<= 3*min(lista_de_valores):
                print(KR,KP*KR,MM,VALOR,justica[:5])
            else:
                pass
        

5 2.5 1800 1.25 ['FW', 'MAD', 'HKA', 'GRX', 'M17']
5 5 1800 1.25 ['FW', 'MAD', 'HKA', 'GRX', 'M17']
5 7.5 1800 1.25 ['FW', 'MAD', 'HKA', 'GRX', 'M17']
5 10 1800 1.25 ['FW', 'MAD', 'HKA', 'GRX', 'M17']
5 12.5 1800 1.25 ['FW', 'MAD', 'HKA', 'GRX', 'M17']
5 15 1800 0.75 ['FW', 'MAD', 'GRX', 'HKA', 'M17']
5 2.5 1850 1.25 ['FW', 'MAD', 'HKA', 'GRX', 'M17']
5 5 1850 1.25 ['FW', 'MAD', 'HKA', 'GRX', 'M17']
5 7.5 1850 1.25 ['FW', 'MAD', 'HKA', 'GRX', 'M17']
5 10 1850 1.25 ['FW', 'MAD', 'HKA', 'GRX', 'M17']
5 12.5 1850 1.5 ['FW', 'MAD', 'GRX', 'HKA', 'M17']
5 15 1850 1.5 ['FW', 'MAD', 'GRX', 'HKA', 'M17']
5 2.5 1900 1.25 ['FW', 'MAD', 'HKA', 'GRX', 'M17']
5 5 1900 1.25 ['FW', 'MAD', 'HKA', 'GRX', 'M17']
5 7.5 1900 1.25 ['FW', 'MAD', 'HKA', 'GRX', 'M17']
5 10 1900 1.25 ['FW', 'MAD', 'HKA', 'GRX', 'M17']
5 12.5 1900 1.5 ['FW', 'MAD', 'GRX', 'HKA', 'M17']
5 15 1900 1.5 ['FW', 'MAD', 'GRX', 'HKA', 'M17']
5 2.5 1950 1.25 ['FW', 'MAD', 'HKA', 'GRX', 'M17']
5 5 1950 1.25 ['FW', 'MAD', 'HKA', 'GRX', 'M

9 9 1950 1.25 ['FW', 'MAD', 'HKA', 'GRX', 'M17']
9 13.5 1950 0.75 ['FW', 'MAD', 'GRX', 'HKA', 'M17']
9 18 1950 0.75 ['FW', 'MAD', 'GRX', 'HKA', 'M17']
9 22.5 1950 0.75 ['FW', 'MAD', 'GRX', 'HKA', 'M17']
9 27 1950 1.5 ['FW', 'MAD', 'GRX', 'HKA', 'M17']
9 4.5 2000 1.25 ['FW', 'MAD', 'HKA', 'GRX', 'M17']
9 9 2000 1.25 ['FW', 'MAD', 'HKA', 'GRX', 'M17']
9 13.5 2000 0.75 ['FW', 'MAD', 'GRX', 'HKA', 'M17']
9 18 2000 0.75 ['FW', 'MAD', 'GRX', 'HKA', 'M17']
9 22.5 2000 0.75 ['FW', 'MAD', 'GRX', 'HKA', 'M17']
9 27 2000 1.75 ['FW', 'MAD', 'GRX', 'M17', 'HKA']
9 4.5 10000 1.25 ['FW', 'MAD', 'HKA', 'GRX', 'M17']
9 9 10000 1.25 ['FW', 'MAD', 'HKA', 'GRX', 'M17']
9 13.5 10000 0.75 ['FW', 'MAD', 'GRX', 'HKA', 'M17']
9 18 10000 0.75 ['FW', 'MAD', 'GRX', 'HKA', 'M17']
9 22.5 10000 0.75 ['FW', 'MAD', 'GRX', 'HKA', 'M17']
9 27 10000 1.75 ['FW', 'MAD', 'GRX', 'M17', 'HKA']
10 5.0 1800 1.25 ['FW', 'MAD', 'HKA', 'GRX', 'M17']
10 10 1800 1.25 ['FW', 'MAD', 'HKA', 'GRX', 'M17']
10 15.0 1800 0.75 ['FW', 'MAD',

KeyboardInterrupt: 

In [556]:
elo(cblol1)
elo(cblol2)
elo(cls1)
elo(cls2)
elo(eu_lcs1,)
elo(eu_lcs2)
elo(lck1)
elo(lck2)
elo(lcl1)
elo(lcl2)
elo(ljl1)
elo(ljl2)
elo(lln1)
elo(lln2)
elo(lms1)
elo(lms2)
elo(lpl1)
elo(lpl2)
elo(na_lcs1)
elo(na_lcs2)
elo(opl1)
elo(opl2)
elo(seatour1)
elo(seatour2)
elo(tcl1)
elo(tcl2)
elo(vcs1)
elo(vcs2)
elo(msi1)
elo(msi2)
elo(msi3)
elo(msi4)
elo(mundial1)
elo(mundial2)
elo(mundial3)

por_vit()

KeyboardInterrupt: 

In [815]:
times[times.Sigla.isin(times_mundial_quartas)].sort_values(by=['Força atualizada'],ascending = False).round(2)

,Sigla,Região,Força Inicial,Força atualizada,Porcentagem de Vitorias
Número,,,,,
1,KT,KR,1850,1850,-1.36
3,AFS,KR,1850,1850,-0.51
11,RNG,CN,1800,1800,-0.02
12,IG,CN,1800,1800,-1.11
13,EDG,CN,1800,1800,-0.30
33,FNC,EU,1800,1800,-1.04
35,G2,EU,1800,1800,1.59
44,C9,NA,1800,1800,-0.38


In [814]:
times[times.Região=='BR'].sort_values(by=['Força atualizada'],ascending = False).round(2)

,Sigla,Região,Força Inicial,Força atualizada,Porcentagem de Vitorias
Número,,,,,
83,CNB,BR,1600,1600,1.46
84,FLA,BR,1600,1600,-0.46
85,KBM,BR,1600,1600,-2.35
86,IDM,BR,1600,1600,2.58
87,VK,BR,1600,1600,1.14
88,ITZ,BR,1600,1600,1.12
89,PRG,BR,1600,1600,-0.70
90,RED,BR,1600,1600,0.81


In [842]:
times.sort_values(by=['Força atualizada'],ascending = False).head(16).round(2)

,Sigla,Região,Força Inicial,Força atualizada,Porcentagem de Vitorias
Número,,,,,
2,GEN,KR,1850,3289.34,0.29
3,AFS,KR,1850,2224.32,0.92
42,H2K,EU,1800,1800.00,0.24
34,VIT,EU,1800,1800.00,0.01
26,MAD,TWHKMA,1800,1800.00,-1.38
27,GRX,TWHKMA,1800,1800.00,-1.94
28,JT,TWHKMA,1800,1800.00,-0.16
29,HKA,TWHKMA,1800,1800.00,-0.37
30,M17,TWHKMA,1800,1800.00,-0.54


In [412]:
previsao('FNC','RNG')

A probabilidade de FNC ganhar é de [0.5]%
A probabilidade de RNG ganhar é de [0.5]%


In [813]:
times[times.Região=='CN'].sort_values(by=['Força atualizada'],ascending = False).round(2)

,Sigla,Região,Força Inicial,Força atualizada,Porcentagem de Vitorias
Número,,,,,
11,RNG,CN,1800,1800,-0.02
12,IG,CN,1800,1800,-1.11
13,EDG,CN,1800,1800,-0.30
14,JDG,CN,1800,1800,0.27
15,RW,CN,1800,1800,-0.06
16,TOP,CN,1800,1800,1.33
17,SNG,CN,1800,1800,0.03
18,FPX,CN,1800,1800,0.71
19,LGD,CN,1800,1800,-0.85
